## 1) 필수 라이브러리 임포트

다음 셀은 이 노트북에서 사용하는 주요 라이브러리를 임포트하고 버전을 출력합니다.

In [1]:
import sys
import platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pathlib import Path
import joblib

print('Python:', sys.version.splitlines()[0])
print('Platform:', platform.platform())
print('pandas', pd.__version__)
print('numpy', np.__version__)
import statsmodels
print('statsmodels', statsmodels.__version__)
print('matplotlib', plt.matplotlib.__version__)
print('seaborn', sns.__version__)


Python: 3.9.6 (default, Nov 11 2024, 03:15:38) 
Platform: macOS-15.1-arm64-arm-64bit
pandas 2.3.3
numpy 2.0.2
statsmodels 0.14.6
matplotlib 3.9.4
seaborn 0.13.2


## 2) 원시 데이터 로드 (CSV)

워크스페이스의 원본 CSV 파일을 읽습니다. 파일이 여러 개인 경우 병합 예시도 포함합니다.

In [2]:
from pathlib import Path

base = Path('/Users/lopi/Downloads/pj/가치 산정 모델')
raw_file = base / 'master_combined_with_rate_kr.csv'
assert raw_file.exists(), f'Missing {raw_file}'

# 읽기
df_raw = pd.read_csv(raw_file, encoding='utf-8-sig')
print('Raw shape:', df_raw.shape)
df_raw.head()


Raw shape: (4748, 44)


,전용면적(㎡),대지권면적(㎡),계약일,거래금액(만원),층,거래유형,관내,PNU,매수자_M,매수자_개인,...,세대수(세대),공원_최단거리,공원_개수,교육_최단거리,교육_개수,유통_최단거리,유통_개수,계약일_파싱,계약월,기준금리(연%)
0,42.00,20.24,2024-01-29,16850,4,중개거래,1,1150010300100560347,0,0,...,NaN,0.53991,2.0,0.82017,1.0,24348.0,1.0,2024-01-29,2024/01,3.5
1,28.96,24.23,2024-01-29,14000,4,직거래,0,1150010300100460195,0,0,...,NaN,0.41431,4.0,1.15973,2.0,27136.0,2.0,2024-01-29,2024/01,3.5
2,28.53,13.35,2024-01-29,8000,4,직거래,0,1150010300100560323,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-29,2024/01,3.5
3,32.88,22.00,2024-01-28,16500,3,중개거래,1,1150010300111290023,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-28,2024/01,3.5
4,55.90,24.57,2024-01-28,19600,3,중개거래,1,1150010300100590085,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-28,2024/01,3.5


## 3) 컬럼 표준화 및 데이터 타입 지정

원본 컬럼명 공백 제거, 숫자 컬럼 강제 변환 예시를 수행합니다.

In [3]:
# 컬럼명 정리

df = df_raw.copy()
# 간단한 정리: 앞뒤 공백 제거
DF_COLS_OLD = df.columns.tolist()
DF_COLS_NEW = [c.strip() for c in DF_COLS_OLD]
df.columns = DF_COLS_NEW

# 숫자 컬럼 예시 변환
num_candidates = ['거래금액(만원)', '전용면적(㎡)', '대지권면적(㎡)', '연면적(㎡)']
for c in num_candidates:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c].astype(str).str.replace(',',''), errors='coerce')

print('After type coercion:')
print(df[num_candidates].dtypes)


After type coercion:
거래금액(만원)      int64
전용면적(㎡)     float64
대지권면적(㎡)    float64
연면적(㎡)      float64
dtype: object


## 4) 결측치 처리 및 중복 제거

중요 컬럼 결측치 처리 전략을 적용하고 전후 행수를 비교합니다.

In [4]:
before = len(df)
# 예: 거래금액/전용면적 결측은 제거
core_cols = ['거래금액(만원)', '전용면적(㎡)']
mask = df[core_cols].notna().all(axis=1)
df = df[mask].copy()
print('Rows before:', before, 'after removing core-nulls:', len(df))

# 중복 제거
before_dup = len(df)
df = df.drop_duplicates()
print('Dropped duplicates:', before_dup - len(df))


Rows before: 4748 after removing core-nulls: 4748
Dropped duplicates: 51


## 5) 날짜 파싱 및 파생변수 생성

'계약일'과 '전월세_최근계약일'을 파싱하고 `Time_Gap`을 만듭니다.

In [5]:
'''
df['계약일_parsed'] = pd.to_datetime(df['계약일'], errors='coerce')
if '전월세_최근계약일' in df.columns:
    df['전월세_최근계약일_parsed'] = pd.to_datetime(df['전월세_최근계약일'], errors='coerce')
    df['Time_Gap'] = (df['계약일_parsed'] - df['전월세_최근계약일_parsed']).dt.days
else:
    df['Time_Gap'] = np.nan

print('Parsed contract dates, Time_Gap NA count:', df['Time_Gap'].isna().sum())
'''

"\ndf['계약일_parsed'] = pd.to_datetime(df['계약일'], errors='coerce')\nif '전월세_최근계약일' in df.columns:\n    df['전월세_최근계약일_parsed'] = pd.to_datetime(df['전월세_최근계약일'], errors='coerce')\n    df['Time_Gap'] = (df['계약일_parsed'] - df['전월세_최근계약일_parsed']).dt.days\nelse:\n    df['Time_Gap'] = np.nan\n\nprint('Parsed contract dates, Time_Gap NA count:', df['Time_Gap'].isna().sum())\n"

## 6) 면적·단위 변환 및 가격 지표 계산

전용면적(㎡)을 평으로 환산하고 `price_per_pyeong`를 계산합니다.

In [6]:
M2_PER_PYEONG = 3.305785
if '전용면적(㎡)' in df.columns:
    df['전용면적_평'] = df['전용면적(㎡)'] / M2_PER_PYEONG

if '거래금액(만원)' in df.columns and '전용면적_평' in df.columns:
    df['price_per_pyeong'] = df['거래금액(만원)'] / df['전용면적_평']

print(df[['전용면적(㎡)','전용면적_평','거래금액(만원)','price_per_pyeong']].head())


   전용면적(㎡)     전용면적_평  거래금액(만원)  price_per_pyeong
0    42.00  12.705000     16850       1326.249458
1    28.96   8.760400     14000       1598.100483
2    28.53   8.630325      8000        926.963898
3    32.88   9.946200     16500       1658.924954
4    55.90  16.909751     19600       1159.094562


## 7) 통화/가격 전처리 및 로그변환

거래금액 숫자화 및 `ln_거래금액` 생성. 0 또는 음수는 제거합니다.

In [7]:
price_cols = ['거래금액(만원)','거래금액','거래금액_원']
price_col = next((c for c in price_cols if c in df.columns), None)
assert price_col is not None, '거래금액 컬럼이 필요합니다'

df[price_col] = pd.to_numeric(df[price_col], errors='coerce')
before_price = len(df)
df = df[df[price_col].notna() & (df[price_col] > 0)].copy()
print('Removed non-positive/NaN prices:', before_price - len(df))

df['ln_거래금액'] = np.log(df[price_col])
print('ln_거래금액 NaN or inf:', df['ln_거래금액'].isna().sum())


Removed non-positive/NaN prices: 0
ln_거래금액 NaN or inf: 0


## 8) 평당 단가 기준 이상치 제거 (0.5% 기준)

평당 단가를 계산하고 하·상위 0.5%를 제거합니다. 제거된 행은 별도 CSV로 저장합니다.

In [8]:
# 평당 단가 계산(이미 계산되어 있을 수 있음)
if 'price_per_pyeong' not in df.columns:
    df['price_per_pyeong'] = df[price_col] / (df['전용면적(㎡)'] / M2_PER_PYEONG)

lower = df['price_per_pyeong'].quantile(0.005)
upper = df['price_per_pyeong'].quantile(0.995)
print('thresholds:', lower, upper)

mask_keep = (df['price_per_pyeong'] >= lower) & (df['price_per_pyeong'] <= upper)
removed_pp = df[~mask_keep].copy()
kept_pp = df[mask_keep].copy()

out_removed = base / 'master_combined_with_rate_kr_pp_removed.csv'
out_filtered = base / 'master_combined_with_rate_kr_ppfiltered.csv'
removed_pp.to_csv(out_removed, index=False, encoding='utf-8-sig')
kept_pp.to_csv(out_filtered, index=False, encoding='utf-8-sig')

print('Removed by pp outlier:', len(removed_pp))
print('Kept after pp filter:', len(kept_pp))

# 샘플 확인: 상·하위 5개
print('Top 5 high price_per_pyeong:')
print(removed_pp.sort_values('price_per_pyeong', ascending=False).head()[['전용면적(㎡)','거래금액(만원)','price_per_pyeong']])
print('Top 5 low price_per_pyeong:')
print(removed_pp.sort_values('price_per_pyeong', ascending=True).head()[['전용면적(㎡)','거래금액(만원)','price_per_pyeong']])


thresholds: 554.7023182142681 4978.757993663126
Removed by pp outlier: 48
Kept after pp filter: 4649
Top 5 high price_per_pyeong:
      전용면적(㎡)  거래금액(만원)  price_per_pyeong
54      15.74     26900       5649.657973
1506    21.65     36900       5634.340254
1494    23.82     39900       5537.398048
1341    20.46     33900       5477.327053
55      16.29     26900       5458.908318
Top 5 low price_per_pyeong:
      전용면적(㎡)  거래금액(만원)  price_per_pyeong
4010    26.16      2000        252.735856
2490    59.90      5600        309.055025
409     35.34      4000        374.169213
1114    77.64     10000        425.783745
1249    59.89      8000        441.580898


## 9-1) 변수 선택 결과 해석 (실무 가이드)

## 9) VIF 계산 및 단계적 변수선택 (Stepwise AIC)

수치형 후보 변수에서 VIF를 반복적으로 계산해 VIF >= 10인 변수를 제거한 뒤, AIC 기준 전진 선택을 수행합니다.

In [9]:
# 10) VIF 반복 제거 + 단계적 선택(AIC) 및 투명한 로깅/시각화

# 목적: 어떤 변수가 왜 제거됐는지(각 반복의 VIF 값)와
# 어느 변수가 어떤 근거(AIC 개선)로 선택되었는지를 표/그래프로 남깁니다.

import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

base = Path('/Users/lopi/Downloads/pj')
images = base / 'report' / 'images'
images.mkdir(parents=True, exist_ok=True)

# 데이터 준비 (pp 필터 이후 우선 사용)
if 'kept_pp' in globals():
    df_model = kept_pp.copy()
elif 'kept' in globals():
    df_model = kept.copy()
else:
    df_model = df.copy()

# 종속변수
price_col = next((c for c in ['거래금액(만원)','거래금액','거래금액_원'] if c in df_model.columns), None)
if 'ln_거래금액' not in df_model.columns:
    df_model['ln_거래금액'] = np.log(pd.to_numeric(df_model[price_col], errors='coerce'))

y = df_model['ln_거래금액'].dropna()
DF = df_model.loc[y.index].copy()
# --- 비선형성 및 교차항 추가 (건축연령^2, 전용면적x층, 건축연령x매수자_법인) ---
try:
    # 건축연령 제곱항
    if '건축연령' in DF.columns:
        DF['건축연령_sq'] = DF['건축연령'].astype(float) ** 2
        print('Added 건축연령_sq')
    # 전용면적 평 변환(없을 경우 시도)
    try:
        if '전용면적_평' not in DF.columns and '전용면적(㎡)' in DF.columns:
            M2_PER_PYEONG = 3.305785
            DF['전용면적_평'] = DF['전용면적(㎡)'].astype(float) / M2_PER_PYEONG
            print('Computed 전용면적_평')
    except Exception:
        pass
    # 전용면적 x 층 (면적과 층 상호작용)
    if '전용면적_평' in DF.columns and '층' in DF.columns:
        DF['area_floor_inter'] = DF['전용면적_평'].astype(float) * DF['층'].astype(float)
        print('Added area_floor_inter')
    # 건축연령 x 매수자_법인 (법인 플래그가 있거나 텍스트로 표시되는 경우 처리)
    if '건축연령' in DF.columns and '매수자_법인' in DF.columns:
        try:
            if DF['매수자_법인'].dtype == object:
                DF['매수자_법인_flag'] = DF['매수자_법인'].astype(str).str.contains('법인').astype(int)
            else:
                DF['매수자_법인_flag'] = DF['매수자_법인'].fillna(0).astype(int)
            DF['age_buycorp_inter'] = DF['건축연령'].astype(float) * DF['매수자_법인_flag'].astype(float)
            print('Added age_buycorp_inter')
        except Exception:
            pass
except Exception as e:
    print('Feature engineering failed:', e)

# 후보 변수 선정 (수치형, 파생/식별자 제외)
exclude = set(['ln_거래금액','거래금액(만원)','거래금액','거래금액_원','price_per_pyeong','계약일_parsed','전월세_최근계약일_parsed','계약일'])
num_cols = DF.select_dtypes(include=[np.number]).columns.tolist()
candidates = [c for c in num_cols if c not in exclude]
# 최소 데이터 가용성 필터 (50% 이상 관측치)
candidates = [c for c in candidates if DF[c].notna().sum() > (0.5 * len(DF))]
print('Initial numeric candidates:', len(candidates))

from statsmodels.stats.outliers_influence import variance_inflation_factor

def compute_vif(frame, features):
    X = frame[features].fillna(0).astype(float)
    Xc = sm.add_constant(X)
    vifs = []
    for i in range(Xc.shape[1]):
        try:
            v = variance_inflation_factor(Xc.values, i)
        except Exception:
            v = np.nan
        vifs.append(v)
    cols = ['const'] + list(features)
    return pd.DataFrame({'feature': cols, 'VIF': vifs})

# -- 기록용 구조체
vif_snapshots = []    # 각 반복에서의 VIF 테이블
drop_log = []         # (iter, dropped_var, dropped_vif)

# 반복적 VIF 제거 (threshold=10)
features = candidates.copy()
iter_no = 0
while True:
    iter_no += 1
    if len(features) == 0:
        break
    vif_df = compute_vif(DF, features)
    vif_df = vif_df[vif_df['feature']!='const'].copy()
    vif_df_sorted = vif_df.sort_values('VIF', ascending=False)
    vif_snapshots.append((iter_no, vif_df_sorted.copy()))
    max_vif = vif_df_sorted['VIF'].max()
    if pd.isna(max_vif):
        break
    if max_vif >= 10.0 and len(features) > 1:
        drop = vif_df_sorted.iloc[0]['feature']
        drop_v = float(vif_df_sorted.iloc[0]['VIF'])
        print(f'Iter {iter_no}: dropping {drop} (VIF={drop_v:.2f})')
        drop_log.append({'iter': iter_no, 'feature': drop, 'VIF': drop_v})
        features.remove(drop)
    else:
        print(f'Iter {iter_no}: no VIF >=10 (max_vif={max_vif:.2f})')
        break

print('Features after VIF pruning:', len(features))

# 저장: VIF 스냅샷과 drop_log
# 합쳐서 보기 좋게 저장
all_vif_frames = []
for it, df_v in vif_snapshots:
    df_v = df_v.copy()
    df_v['iter'] = it
    all_vif_frames.append(df_v)
if all_vif_frames:
    vif_history = pd.concat(all_vif_frames, ignore_index=True)
    vif_history.to_csv(base / 'vif_history.csv', index=False, encoding='utf-8-sig')
pd.DataFrame(drop_log).to_csv(base / 'vif_drop_log.csv', index=False, encoding='utf-8-sig')

# 시각화: 초기/최종 VIF 바플롯
if vif_snapshots:
    # initial
    init_iter, init_vif = vif_snapshots[0]
    fig, ax = plt.subplots(figsize=(8, max(4, len(init_vif)/4)))
    sns.barplot(data=init_vif.sort_values('VIF', ascending=False), x='VIF', y='feature', ax=ax, palette='viridis')
    ax.set_title('Initial VIF (sorted)')
    fig.tight_layout()
    fig.savefig(images / 'vif_initial.png', dpi=150)
    plt.close(fig)

    # final (last snapshot)
    final_iter, final_vif = vif_snapshots[-1]
    fig, ax = plt.subplots(figsize=(8, max(4, len(final_vif)/4)))
    sns.barplot(data=final_vif.sort_values('VIF', ascending=False), x='VIF', y='feature', ax=ax, palette='magma')
    ax.set_title(f'VIF after pruning (iter {final_iter})')
    fig.tight_layout()
    fig.savefig(images / 'vif_pruned.png', dpi=150)
    plt.close(fig)



Added 건축연령_sq
Added area_floor_inter
Added age_buycorp_inter
Initial numeric candidates: 33
Iter 1: no VIF >=10 (max_vif=1.64)
Features after VIF pruning: 33


In [10]:
# 단계적 전진 선택 (AIC) — 전진 과정과 AIC 기록
X_all = DF[features].fillna(0).astype(float)
y = DF['ln_거래금액']

def aic_of_features(feat_list):
    if len(feat_list) == 0:
        model = sm.OLS(y, np.ones((len(y),1))).fit()
        return model.aic
    Xf = sm.add_constant(X_all[feat_list])
    model = sm.OLS(y, Xf).fit()
    return model.aic

selected = []
current_aic = aic_of_features([])
step_history = []
step_no = 0
step_history.append({'step': step_no, 'action': 'intercept', 'feature': None, 'AIC': current_aic})
print('Intercept-only AIC:', current_aic)

remaining = set(features)
improved = True
while improved and remaining:
    improved = False
    best_candidate = None
    best_aic = current_aic
    for cand in sorted(remaining):
        try:
            aic = aic_of_features(selected + [cand])
        except Exception:
            aic = np.inf
        if aic < best_aic:
            best_aic = aic
            best_candidate = cand
    if best_candidate is not None:
        step_no += 1
        selected.append(best_candidate)
        remaining.remove(best_candidate)
        current_aic = best_aic
        improved = True
        step_history.append({'step': step_no, 'action': 'add', 'feature': best_candidate, 'AIC': current_aic})
        print('Step', step_no, 'Added', best_candidate, 'AIC=', current_aic)

# 후진 검사 (단순): 제거로 AIC 개선되면 제거
improved = True
while improved and len(selected) > 1:
    improved = False
    for feat in selected.copy():
        trial = [f for f in selected if f != feat]
        aic = aic_of_features(trial)
        if aic < current_aic:
            step_no += 1
            selected.remove(feat)
            current_aic = aic
            improved = True
            step_history.append({'step': step_no, 'action': 'remove', 'feature': feat, 'AIC': current_aic})
            print('Step', step_no, 'Removed', feat, 'improved AIC to', current_aic)
            break

# 기록 저장
pd.DataFrame(step_history).to_csv(base / 'stepwise_history.csv', index=False, encoding='utf-8-sig')
(base / 'selected_features.txt').write_text('\n'.join(selected), encoding='utf-8')
print('Selected features count:', len(selected))
print(selected)

# AIC 경과 시각화
sh_df = pd.DataFrame(step_history)
if not sh_df.empty:
    plt.figure(figsize=(8,4))
    plt.plot(sh_df['step'], sh_df['AIC'], marker='o')
    for _, r in sh_df.iterrows():
        if r['action'] != 'intercept':
            plt.text(r['step'], r['AIC'], f"{r['action']}:{r['feature']}", fontsize=8, rotation=25)
    plt.gca().invert_yaxis()
    plt.xlabel('Step')
    plt.ylabel('AIC (lower better)')
    plt.title('Stepwise AIC progression')
    plt.tight_layout()
    plt.savefig(images / 'stepwise_aic.png', dpi=150)
    plt.close()



Intercept-only AIC: 3346.190973254579
Step 1 Added 건축연령 AIC= 2226.5558202081556
Step 2 Added 전용면적(㎡) AIC= -145.51177361802365
Step 3 Added 건축연령_sq AIC= -1182.5595425233369
Step 4 Added area_floor_inter AIC= -1376.5752455770562
Step 5 Added 관내 AIC= -1500.4756327349478
Step 6 Added 전월세_평균_보증금(만원) AIC= -1621.4714401918427
Step 7 Added 매도자_법인 AIC= -1696.4607744229634
Step 8 Added 기준금리(연%) AIC= -1759.1173563064258
Step 9 Added 전월세_평균_월세(만원) AIC= -1807.6368161228766
Step 10 Added 전월세_건수 AIC= -1843.9946848601758
Step 11 Added 공원_최단거리 AIC= -1864.3445917973277
Step 12 Added 매수자_개인 AIC= -1874.8148218300157
Step 13 Added 층 AIC= -1882.2635503342754
Step 14 Added 교육_최단거리 AIC= -1885.4526430714832
Step 15 Added 매수자_법인 AIC= -1888.4774824283486
Step 16 Added age_buycorp_inter AIC= -1909.8278574799697
Step 17 Added 매도자_개인 AIC= -1913.0436352243087
Step 18 Added 거래유형_중개거래 AIC= -1915.771396987344
Step 19 Added 유통_최단거리 AIC= -1916.9587462223317
Step 20 Added 매도자_M AIC= -1918.0401970352232
Step 21 Removed 매도자

In [11]:
# 최종 모델 적합 및 계수 신뢰구간 시각화
if len(selected) > 0:
    X_final = sm.add_constant(X_all[selected])
else:
    X_final = np.ones((len(y),1))
model_final = sm.OLS(y, X_final).fit()
print(model_final.summary())

coef = model_final.params.copy()
conf = model_final.conf_int()
coef_df = pd.DataFrame({'coef': coef.values, 'ci_low': conf[0].values, 'ci_high': conf[1].values}, index=coef.index)
coef_df.to_csv(base / 'model_coefficients.csv', encoding='utf-8-sig')

# coef plot (exclude intercept if present)
coef_plot = coef_df.loc[[c for c in coef_df.index if c != 'const']]
if not coef_plot.empty:
    fig, ax = plt.subplots(figsize=(8, max(4, len(coef_plot)/4)))
    coef_plot = coef_plot.sort_values('coef')
    ax.errorbar(coef_plot['coef'], coef_plot.index, xerr=[coef_plot['coef']-coef_plot['ci_low'], coef_plot['ci_high']-coef_plot['coef']], fmt='o')
    ax.axvline(0, color='gray', linestyle='--')
    ax.set_xlabel('Coefficient')
    ax.set_title('Coefficients and 95% CI')
    fig.tight_layout()
    fig.savefig(images / 'coef_ci.png', dpi=150)
    plt.close(fig)

# --- Diagnostics: Condition Number, final VIF, Jarque-Bera, and optional trimming ---
import numpy as _np
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.outliers_influence import variance_inflation_factor as _vif
# Condition Number
try:
    X_for_cond = X_final.values if hasattr(X_final, 'values') else _np.asarray(X_final)
    cond_no = _np.linalg.cond(X_for_cond)
    print(f'Condition Number: {cond_no:.2e}')
    if cond_no > 1e6:
        print('WARNING: Condition number > 1e6 — severe multicollinearity.')
except Exception as e:
    print('Condition number compute failed:', e)

# Final VIF (including const) — helpful to see remaining collinearity
try:
    Xv = X_final.fillna(0).astype(float)
    vif_vals = []
    for i in range(Xv.shape[1]):
        try:
            vif_vals.append(_vif(Xv.values, i))
        except Exception:
            vif_vals.append(_np.nan)
    vif_df_final = pd.DataFrame({'feature': Xv.columns, 'VIF': vif_vals})
    print('Final VIF:')
    print(vif_df_final.sort_values('VIF', ascending=False).to_string(index=False))
except Exception as e:
    print('Final VIF compute failed:', e)

# Residual normality: Jarque-Bera
try:
    jb_stat, jb_pvalue, skew, kurtosis = jarque_bera(model_final.resid)
    print(f'Jarque-Bera stat: {jb_stat:.3f}, p-value: {jb_pvalue:.3f}')
    print(f'Skew: {skew:.3f}, Kurtosis: {kurtosis:.3f}')
except Exception as e:
    print('JB test failed:', e)

# Optional: Trim extreme ln_거래금액 (top/bottom 1%) and compare model
try:
    low = DF['ln_거래금액'].quantile(0.01)
    high = DF['ln_거래금액'].quantile(0.99)
    print('Trimming ln_거래금액 bounds (1%/99%):', low, high)
    mask_trim = DF['ln_거래금액'].between(low, high)
    DF_trim = DF[mask_trim].copy()
    if len(DF_trim) < len(DF):
        X_trim = X_final.loc[DF_trim.index] if hasattr(X_final, 'loc') else sm.add_constant(X_all[selected]).loc[DF_trim.index]
        model_trim = sm.OLS(DF_trim['ln_거래금액'], X_trim).fit()
        print('Trimmed model: N=', len(DF_trim), 'AIC=', model_trim.aic)
        try:
            jb_t = jarque_bera(model_trim.resid)
            print('Trimmed Jarque-Bera p-value:', jb_t[1])
        except Exception:
            pass
    else:
        print('No trimming applied (no extremes found)')
except Exception as e:
    print('Trimming/refit failed:', e)

                            OLS Regression Results                            
Dep. Variable:                ln_거래금액   R-squared:                       0.680
Model:                            OLS   Adj. R-squared:                  0.679
Method:                 Least Squares   F-statistic:                     547.6
Date:                Wed, 04 Feb 2026   Prob (F-statistic):               0.00
Time:                        20:22:26   Log-Likelihood:                 979.50
No. Observations:                4649   AIC:                            -1921.
Df Residuals:                    4630   BIC:                            -1799.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 9.7390      0.03

### 최종 헤도닉 식
ln_거래금액 = 9.7389907 + (모든 항 합)

(-0.06856765)*건축연령
0.01590703*전용면적(㎡)
0.001155161*건축연령_sq
0.002623821*area_floor_inter
0.05232206*관내
0.000003527508*전월세_평균_보증금(만원)
(-0.03412908)*기준금리(연%)
0.001204104*전월세_평균_월세(만원)
0.003531350*전월세_건수
0.04841015*공원_최단거리
(-0.009736569)*층
(-0.01531285)*교육_최단거리
(-0.26885610)*매수자_법인
0.01008808*age_buycorp_inter
0.12695804*매도자_개인
0.02565999*거래유형_중개거래
(-0.0000002978722)*유통_최단거리
0.10579059*매도자_M

---
예측 원가격은 역변환해서 구합니다:
Predicted_Price = exp( ln_거래금액 )

In [12]:
# 예측 및 잔차
# 모델로부터 예측값을 생성하고 역변환 및 잔차를 계산합니다
try:
    # X_final이 DataFrame/array인지에 따라 predict에 적절한 입력 사용
    preds = model_final.predict(X_final)
    DF['Predicted_ln'] = preds
    DF['Predicted_Price'] = np.exp(DF['Predicted_ln'])
    DF['Actual_Price'] = pd.to_numeric(DF[price_col], errors='coerce')
    DF['Residual'] = DF['Actual_Price'] - DF['Predicted_Price']
except Exception as e:
    print('Prediction failed:', e)
    DF['Predicted_ln'] = np.nan
    DF['Predicted_Price'] = np.nan
    DF['Actual_Price'] = pd.to_numeric(DF.get(price_col, pd.Series(dtype=float)), errors='coerce')
    DF['Residual'] = np.nan



In [15]:
# Bootstrap confidence intervals for coefficients (percentile bootstrap)
B = 500
import time
params = []
# Build X for bootstrap: use selected features if available
X_boot_base = X_all[selected] if 'selected' in globals() and 'X_all' in globals() else (X_all if 'X_all' in globals() else None)
if X_boot_base is None:
    print('Bootstrap skipped: X_all or selected not found')
else:
    Xb = sm.add_constant(X_boot_base)
    y_full = DF['ln_거래금액']
    n = len(y_full)
    np.random.seed(0)
    start = time.time()
    for i in range(B):
        idx = np.random.choice(n, size=n, replace=True)
        try:
            Xs = Xb.iloc[idx] if hasattr(Xb, 'iloc') else Xb[idx]
            ys = y_full.iloc[idx] if hasattr(y_full, 'iloc') else y_full[idx]
            mod = sm.OLS(ys, Xs).fit()
            params.append(mod.params.values)
        except Exception:
            continue
    if len(params) == 0:
        print('No successful bootstrap fits; skipping bootstrap outputs')
    else:
        params = np.vstack(params)
        param_names = Xb.columns
        ci_lower = np.percentile(params, 2.5, axis=0)
        ci_upper = np.percentile(params, 97.5, axis=0)
        boot_std = params.std(axis=0)
        coef_boot_df = pd.DataFrame({'feature': param_names, 'coef': model_final.params.values, 'ci_low_boot': ci_lower, 'ci_high_boot': ci_upper, 'boot_std': boot_std})
        coef_boot_df.to_csv(base / 'model_coefficients_bootstrap.csv', index=False, encoding='utf-8-sig')
        print('Bootstrap finished: saved', base / 'model_coefficients_bootstrap.csv')
        # Display saved CSV in this cell's output for quick inspection
        try:
            df_boot = pd.read_csv(base / 'model_coefficients_bootstrap.csv', encoding='utf-8-sig')
            from IPython.display import display
            print('Bootstrap coefficients (first 20 rows):')
            display(df_boot.head(20))
        except Exception as e:
            print('Failed to read/display bootstrap CSV:', e)
        try:
            cb = coef_boot_df.set_index('feature')
            fig, ax = plt.subplots(figsize=(8, max(4, len(cb)/4)))
            cb_sorted = cb.sort_values('coef')
            ax.errorbar(cb_sorted['coef'], cb_sorted.index, xerr=[cb_sorted['coef']-cb_sorted['ci_low_boot'], cb_sorted['ci_high_boot']-cb_sorted['coef']], fmt='o')
            ax.axvline(0, color='gray', linestyle='--')
            ax.set_xlabel('Coefficient (bootstrap 95% CI)')
            ax.set_title('Bootstrap Coefficients and 95% CI')
            fig.tight_layout()
            fig.savefig(images / 'coef_ci_bootstrap.png', dpi=150)
            plt.close(fig)
        except Exception as e:
            print('Bootstrap plot failed:', e)
    print('Elapsed (s):', time.time()-start)


Bootstrap finished: saved /Users/lopi/Downloads/pj/model_coefficients_bootstrap.csv
Bootstrap coefficients (first 20 rows):


,feature,coef,ci_low_boot,ci_high_boot,boot_std
0,const,9.738991e+00,9.666697e+00,9.807287e+00,3.781234e-02
1,건축연령,-6.856765e-02,-7.184923e-02,-6.523361e-02,1.746149e-03
2,전용면적(㎡),1.590703e-02,1.485995e-02,1.703137e-02,5.703395e-04
3,건축연령_sq,1.155161e-03,1.075625e-03,1.242977e-03,4.560999e-05
4,area_floor_inter,2.623821e-03,1.559468e-03,3.579563e-03,5.250247e-04
5,관내,5.232206e-02,3.117986e-02,7.505160e-02,1.127476e-02
6,전월세_평균_보증금(만원),3.527508e-06,2.609856e-06,4.386209e-06,4.859568e-07
7,기준금리(연%),-3.412908e-02,-4.447129e-02,-2.438552e-02,5.177357e-03
8,전월세_평균_월세(만원),1.204104e-03,8.280597e-04,1.523067e-03,1.776354e-04
9,전월세_건수,3.531350e-03,2.488531e-03,4.595534e-03,5.425420e-04


Elapsed (s): 1.036236047744751


## 부트스트랩 분석 결과
### 변수 안전성_매우 안정적인 변수
* 건축연령(-0.068)
* 전용면적(0.0159)
* 건축연령_sq(0.0011) (파생변수)
* 매수자_법인(-0.26) 
* age_buycorp_inter(파생변수) -> 법인+신축일수록 가격 왜곡 심하다는 의미 (전세사기의 전형적인 패턴)
* area_floor_inter(파생변수) -> 면적+층
### 변수안전성_불안정한 변수
* 층
* 교육_최단거리
* 거래유형_중개거래
---
### 계수 안정성
* 표준편차 매우 낮음
## 👍🏻'세웠던 파생변수들이 통계적으로 매우 탄탕함' 개굿.

In [14]:
# 저장
out_model = base / 'MD1(최종).csv'
DF.to_csv(out_model, index=False, encoding='utf-8-sig')
(base / 'hedonic_model_summary.txt').write_text(model_final.summary().as_text(), encoding='utf-8')

print('Saved:')
print(' -', out_model)
print(' -', base / 'selected_features.txt')
print(' -', base / 'hedonic_model_summary.txt')
print(' - VIF history:', base / 'vif_history.csv')
print(' - VIF drop log:', base / 'vif_drop_log.csv')
print(' - Stepwise history:', base / 'stepwise_history.csv')
print(' - Images in', images)


Saved:
 - /Users/lopi/Downloads/pj/MD1(최종).csv
 - /Users/lopi/Downloads/pj/selected_features.txt
 - /Users/lopi/Downloads/pj/hedonic_model_summary.txt
 - VIF history: /Users/lopi/Downloads/pj/vif_history.csv
 - VIF drop log: /Users/lopi/Downloads/pj/vif_drop_log.csv
 - Stepwise history: /Users/lopi/Downloads/pj/stepwise_history.csv
 - Images in /Users/lopi/Downloads/pj/report/images


## 사용자 시나리오/유즈케이스
## 기대효과 및 향후 확장성